In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xindai/test_split.csv
/kaggle/input/xindai/test_unlabel.csv
/kaggle/input/xindai/xindai/xindai/dev.csv
/kaggle/input/xindai/xindai/xindai/train.csv
/kaggle/input/xindai/xindai/xindai/test.csv
/kaggle/input/sentencebert/sentence-transformers-master/index.rst
/kaggle/input/sentencebert/sentence-transformers-master/NOTICE.txt
/kaggle/input/sentencebert/sentence-transformers-master/setup.cfg
/kaggle/input/sentencebert/sentence-transformers-master/LICENSE
/kaggle/input/sentencebert/sentence-transformers-master/.gitignore
/kaggle/input/sentencebert/sentence-transformers-master/README.md
/kaggle/input/sentencebert/sentence-transformers-master/requirements.txt
/kaggle/input/sentencebert/sentence-transformers-master/setup.py
/kaggle/input/sentencebert/sentence-transformers-master/tests/test_train_stsb.py
/kaggle/input/sentencebert/sentence-transformers-master/tests/test_pretrained_stsb.py
/kaggle/input/sentencebert/sentence-transformers-master/tests/test_multi_process.py
/kaggle/i

In [2]:
!pip install sentence_transformer

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [3]:
import sys 
sys.path.append('/kaggle/input/sentencebert/sentence-transformers-master')
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util

In [4]:
from torch.utils.data import DataLoader
import math
import sys
sys.path.append('/kaggle/input/sentencebert/sentence-transformers-master')
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import pandas as pd
print('---import finished')

---import finished


In [5]:
def sbert_acc():
    import numpy as np
    df_data = pd.read_csv('../input/xindai/test_split.csv', sep='\t')
    df_data = df_data.head(1)
    print(df_data.head(10))

    # Pre-trained cross encoder\
    #model = CrossEncoder('cross-encoder/stsb-distilroberta-base')  # 训练好的相似度计算模型
    #uer/sbert-base-chinese-nli
    model = CrossEncoder('uer/sbert-base-chinese-nli')
    sentence_combinations =df_data[['sent1', 'sent2']].values.tolist()

    # Compute the similarity scores for these combinations
    similarity_scores = model.predict(sentence_combinations, batch_size=1) # 计算相似度得分
    print('---similarity_scores', type(similarity_scores),similarity_scores)

In [6]:
#sbert_acc()

In [7]:
from sklearn.metrics import accuracy_score


In [8]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# model = BertModel.from_pretrained("uer/sbert-base-chinese-nli")
# tokenizer = BertTokenizer.from_pretrained("uer/sbert-base-chinese-nli")
# # sentences = [
# # " 什么的涟漪 ",
# # " 什么的的涟漪",
# # ]
# sentences = ['什么的涟漪', '什么的的涟漪']
# # 初始化字典来存储
# tokens = {'input_ids': [], 'attention_mask': []}
# for sentence in sentences:
#     new_tokens = tokenizer.encode_plus(sentence, max_length=15, truncation=True, padding='max_length', return_tensors='pt')
#     tokens['input_ids'].append(new_tokens['input_ids'][0])
#     tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# # 将张量列表重新格式化为一个张量
# tokens['input_ids'] = torch.stack(tokens['input_ids'])
# tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

# outputs = model(**tokens)
# # print(outputs.keys()) #odict_keys(['last_hidden_state', 'pooler_output'])

# embeddings = outputs.last_hidden_state
# # print(embeddings.shape) #torch.Size([4, 15, 768])

# attention_mask = tokens['attention_mask']
# # print(attention_mask.shape) #torch.Size([4, 15])

# mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
# # print(mask.shape) #torch.Size([4, 15, 768])

# masked_embeddings = embeddings * mask
# # print(masked_embeddings.shape) #torch.Size([4, 15, 768])

# summed = torch.sum(masked_embeddings, 1)
# # print(summed.shape) #torch.Size([4, 768])

# summed_mask = torch.clamp(mask.sum(1), min=1e-9)
# # print(summed_mask.shape) #torch.Size([4, 768])

# mean_pooled = summed / summed_mask
# # print(mean_pooled.shape) #torch.Size([4, 768])

# mean_pooled = mean_pooled.detach().numpy()
# result = cosine_similarity([mean_pooled[0]], mean_pooled[1:])
# print(result) #[[0.9864919 0.39011386 0.29779416]]

In [9]:
def sbert_acc():
    df_data = pd.read_csv('../input/xindai/test_unlabel.csv', names=['sent1','sent2'],sep='\t')
    df_data = df_data.head(10)
    text = df_data[['sent1','sent2']].values.tolist()
    #print('--text',text)
    from transformers import BertTokenizer, BertModel
    from sklearn.metrics.pairwise import cosine_similarity
    import torch

    model = BertModel.from_pretrained("uer/sbert-base-chinese-nli")
    tokenizer = BertTokenizer.from_pretrained("uer/sbert-base-chinese-nli")
    all_result =[]
    for i in range(len(text)):
        sentences = text[i]
        #print('----sentences',sentences)
        # 初始化字典来存储
        #sentences = ['什么的涟漪', '什么的的涟漪']
        tokens = {'input_ids': [], 'attention_mask': []}
        for sentence in sentences:
            new_tokens = tokenizer.encode_plus(sentence, max_length=15, truncation=True, padding='max_length',
                                               return_tensors='pt')
            tokens['input_ids'].append(new_tokens['input_ids'][0])
            tokens['attention_mask'].append(new_tokens['attention_mask'][0])

        # 将张量列表重新格式化为一个张量
        tokens['input_ids'] = torch.stack(tokens['input_ids'])
        tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

        outputs = model(**tokens)
        # print(outputs.keys()) #odict_keys(['last_hidden_state', 'pooler_output'])

        embeddings = outputs.last_hidden_state
        # print(embeddings.shape) #torch.Size([4, 15, 768])

        attention_mask = tokens['attention_mask']
        # print(attention_mask.shape) #torch.Size([4, 15])

        mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        # print(mask.shape) #torch.Size([4, 15, 768])

        masked_embeddings = embeddings * mask
        # print(masked_embeddings.shape) #torch.Size([4, 15, 768])

        summed = torch.sum(masked_embeddings, 1)
        # print(summed.shape) #torch.Size([4, 768])

        summed_mask = torch.clamp(mask.sum(1), min=1e-9)
        # print(summed_mask.shape) #torch.Size([4, 768])

        mean_pooled = summed / summed_mask
        # print(mean_pooled.shape) #torch.Size([4, 768])

        mean_pooled = mean_pooled.detach().numpy()
        result = cosine_similarity([mean_pooled[0]], [mean_pooled[1]])[0][0]
        #print(result)  # [[0.9864919 0.39011386 0.29779416]]
        all_result.append(result)
    df_data['cos']=all_result
    print(df_data.head(10))
    df_data.to_csv('./submit_result.txt', index=False, sep='\t')
    return df_data

In [10]:
df_result = sbert_acc()

Downloading:   0%|          | 0.00/559 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

             sent1          sent2       cos
0         成语中的历史人物      成语有关的历史人物  0.870867
1          黄财神怎样供奉         怎样供奉财神  0.754249
2          进门是餐厅好吗        进门见餐厅好吗  0.914723
3         怎么提高理解力？        怎样提高理解力  0.971642
4      仓鼠用什么磨牙都可以吗  用什么可以代替仓鼠的磨牙棒  0.657281
5         什么杀毒软件好？        什么杀毒软件好  0.931160
6         请问这是哪部动漫      请问这是哪部动漫的  0.977423
7            动漫男图片          动漫图片男  0.889867
8  哥本哈根减肥餐能喝红豆薏米水吗    夏天可以喝薏仁红豆汤吗  0.693385
9         电子书怎么下载？        怎么下载电子书  0.933850


In [11]:
# from sklearn.metrics import accuracy_score
# df_result['sim_flag'] = df_result['cos'].apply(lambda x: 1 if x >= 0.75 else 0)
# y_pre = np.array(df_result['sim_flag'].values.tolist())
# y_true = np.array(df_result['label'].values.tolist())
# print(accuracy_score(y_true, y_pre))

In [12]:
# model_name = 'hfl/chinese-roberta-wwm-ext-large'
# train_batch_size = 16
# num_epochs = 4
# model_save_path = 'test_output'
# logging.basicConfig(format='%(asctime)s - %(message)s',
#   datefmt='%Y-%m-%d %H:%M:%S',
#   level=logging.INFO,
#   handlers=[LoggingHandler()])

# # Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
# word_embedding_model = models.Transformer(model_name)

# # Apply mean pooling to get one fixed sized sentence vector
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
#   pooling_mode_mean_tokens=True,
#   pooling_mode_cls_token=False,
#   pooling_mode_max_tokens=False)

# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# train_samples = []
# dev_samples = []
# test_samples = []

# def load(path):
#   df = pd.read_csv(path)
#   samples = []
#   for idx,item in df.iterrows():
#     samples.append(InputExample(texts=[item['sentence1'], item['sentence2']], label=float(item['label'])))
#     samples = samples[:1000]
#   return samples

# data_dir = '/data/keraszhou/ask_match/data/base_data/xindai/'
# train_samples = load('/kaggle/input/xindai/xindai/train.csv')
# test_samples = load('/kaggle/input/xindai/xindai/dev.csv')
# dev_samples = load( '/kaggle/input/xindai/xindai/test.csv')

# train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
# train_loss = losses.CosineSimilarityLoss(model=model)
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')
# warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

# # Train the model
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#   evaluator=evaluator,
#   epochs=num_epochs,
#   evaluation_steps=1000,
#   warmup_steps=warmup_steps,
#   output_path=model_save_path)

# model = SentenceTransformer(model_save_path)
# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
# test_evaluator(model, output_path=model_save_path)